<a href="https://colab.research.google.com/github/gouthamans14/Transformers_Notebooks/blob/main/NER_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
! pip install transformers datasets

In [123]:
from datasets import load_dataset
import numpy as np
import pandas as pd

In [63]:
data = load_dataset("conll2003")

In [64]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [65]:
data['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [66]:
data['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}

In [67]:
data['train'].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [68]:
data['train'].features['ner_tags'].feature.names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [69]:
from transformers import AutoTokenizer

In [70]:
checkpoint = 'distilbert-base-cased'
tokenizer  = AutoTokenizer.from_pretrained(checkpoint)

In [71]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [72]:
data["train"][0]['tokens']

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [73]:
t =tokenizer(data["train"][0]['tokens'], is_split_into_words=True)

In [74]:
type(t)

transformers.tokenization_utils_base.BatchEncoding

In [75]:
len(t.tokens())

12

In [76]:
idx=0

data['train'][idx]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [77]:
len(data['train'][idx]['tokens'])

9

In [78]:
len(data['train'][idx]['ner_tags'])

9

In [79]:
checkpoint = 'distilbert-base-cased'
tokenizer  = AutoTokenizer.from_pretrained(checkpoint)

In [80]:
idx=0
t = tokenizer(data['train'][idx]["tokens"], is_split_into_words=True)

In [81]:
[]

[]

In [82]:
t.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [83]:
begin2inside = {
  1: 2,
  3: 4,
  5: 6,
  7: 8,
}

In [84]:
data['train']['tokens'][0]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [85]:
def align_target(labels,word_ids):
    aligned_labels=[]
    last_word = None
    for word in word_ids:
        if word is None:
            label=-100
        elif word != last_word:
            label = labels[word]
        else:
            label=labels[word]


        if label in begin2inside:
            label = begin2inside[label]

        aligned_labels.append(label)

        last_word = word

    return aligned_labels


In [86]:
labels = data['train'][idx]['ner_tags']
word_ids = t.word_ids()
aligned_targets = align_target(labels,word_ids)
aligned_targets

[-100, 4, 0, 8, 0, 0, 0, 8, 0, 0, 0, -100]

In [87]:
tokenizer(data["train"][0]['tokens'], is_split_into_words=True).word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [88]:
# save for later
label_names = data["train"].features['ner_tags'].feature.names

In [89]:
def align_targets(labels, word_ids):
  aligned_labels = []
  last_word = None
  for word in word_ids:
    if word is None:
      # it's a token like [CLS]
      label = -100
    elif word != last_word:
      # it's a new word!
      label = labels[word]
    else:
      # it's the same word as before
      label = labels[word]

      # change B-<tag> to I-<tag> if necessary
      if label in begin2inside:
        label = begin2inside[label]

    # add the label
    aligned_labels.append(label)

    # update last word
    last_word = word

  return aligned_labels

In [90]:
# make up a fake input just to test it
words = [
  '[CLS]', 'Ger', '##man', 'call', 'to', 'boycott', 'Micro', '##soft', '[SEP]']
word_ids = [None, 0, 0, 1, 2, 3, 4, 4, None]
labels = [7, 0, 0, 0, 3]
aligned_targets = align_target(labels, word_ids)
aligned_labels = [label_names[t] if t >= 0 else None for t in aligned_targets]
for x, y in zip(words, aligned_labels):
  print(f"{x}\t{y}")

[CLS]	None
Ger	I-MISC
##man	I-MISC
call	O
to	O
boycott	O
Micro	I-ORG
##soft	I-ORG
[SEP]	None


In [91]:
aligned_labels= [label_names[t] if t>=0 else None for t in aligned_targets]
for x, y in  zip(t.tokens(), aligned_labels):
    print(f"{x} \t {y}")

[CLS] 	 None
EU 	 I-MISC
rejects 	 I-MISC
German 	 O
call 	 O
to 	 O
boycott 	 I-ORG
British 	 I-ORG
la 	 None


In [92]:
def tokenize_fn(batch):
    tokenized_inputs = tokenizer(batch['tokens'],truncation=True, is_split_into_words=True)

    labels_batch = batch['ner_tags']
    aligned_labels_batch =[]
    for i, labels in enumerate(labels_batch):
        word_ids = tokenized_inputs.word_ids(i)
        aligned_labels_batch.append(align_target(labels,word_ids))
    tokenized_inputs['labels']= aligned_labels_batch
    return tokenized_inputs

In [93]:
# labels_batch = data['train']['ner_tags']
# aligned_labels_batch =[]
# tokenized_inputs = tokenizer(data['train']['tokens'],truncation=True, is_split_into_words=True)
# for i, labels in enumerate(labels_batch[:2]):
#     word_ids = tokenized_inputs.word_ids(i)
#     aligned_labels_batch.append(align_target(labels,word_ids))
#     print(aligned_labels_batch)

In [94]:
tokenized_datasets=data.map(
    tokenize_fn,
    batched=True,
    remove_columns=data['train'].column_names

)

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [95]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

## Data Collator

* consider our text preprocessing review: pad, truncate, Torch tesnors
* We haven't been explicitly doing all of these steps!
* We dont want to pad relative to whole datset, just current batch
* All of the above done by "Data Collator"

Note:
* For previous task, ie. for sequence classification the data collator is already been called in trainer api

* for current task, this does not work, so we have to define one explicitly.

### Code:

In [96]:
from transformers import DataCollatorForTokenClassification

In [97]:
data_collator = DataCollatorForTokenClassification(tokenizer= tokenizer)

In [98]:
tokenized_datasets["train"][0:2]

{'input_ids': [[101,
   7270,
   22961,
   1528,
   1840,
   1106,
   21423,
   1418,
   2495,
   12913,
   119,
   102],
  [101, 1943, 14428, 102]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]],
 'labels': [[-100, 4, 0, 8, 0, 0, 0, 8, 0, 0, 0, -100], [-100, 2, 2, -100]]}

### A data collator can work with only list

In [99]:
ls = [tokenized_datasets["train"][i] for i in range(0,2)]

In [100]:
batch = data_collator(ls)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [101]:
batch

{'input_ids': tensor([[  101,  7270, 22961,  1528,  1840,  1106, 21423,  1418,  2495, 12913,
           119,   102],
        [  101,  1943, 14428,   102,     0,     0,     0,     0,     0,     0,
             0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]), 'labels': tensor([[-100,    4,    0,    8,    0,    0,    0,    8,    0,    0,    0, -100],
        [-100,    2,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])}

In [102]:
batch['labels']

tensor([[-100,    4,    0,    8,    0,    0,    0,    8,    0,    0,    0, -100],
        [-100,    2,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

# Computing Metrics for Token Classification

In [103]:
! pip install seqeval

In [104]:
from datasets import load_metric
metric = load_metric("seqeval")
metric.compute(predictions=[[0,0,0]] ,references =[[0,0,1]])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:5

{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.6666666666666666}

In [105]:
metric.compute(predictions=[['A','A','A']] ,references =[['B','B','B']])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: A seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'_': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 3},
 'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.0}

In [106]:
metric.compute(predictions=[['O','O','I-ORG','B-MISC']],references=[['O','B-ORG','I-ORG','B-MISC']])

{'MISC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ORG': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'overall_precision': 0.5,
 'overall_recall': 0.5,
 'overall_f1': 0.5,
 'overall_accuracy': 0.75}

In [107]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [120]:
def compute_metrics(logits_and_labels):
    logits, labels= logits_and_labels
    preds = np.argmax(logits, axis=-1)

    str_labels = [[label_names[t] for t in label if t!=-100] for label in labels]

    str_preds = [[label_names[p] for p, t in zip(pred, targ) if t!=-100] for pred, targ in zip(preds, labels)]

    the_metrics = metric.compute(predictions=str_preds, references = str_labels)
    return{
        "precision": the_metrics['overall_precision'],
        "recall": the_metrics['overall_recall'],
        "f1": the_metrics['overall_f1'],
        "accuracy": the_metrics["overall_accuracy"]
    }

## Model

In [109]:
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [110]:
id2label = {k:v for k, v in enumerate(label_names)}

In [111]:
label2id = {v:k for k,v in id2label.items()}

In [112]:
from transformers import AutoModelForTokenClassification

In [113]:
model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label = id2label,
    label2id = label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [114]:
from transformers import TrainingArguments

In [115]:
training_args = TrainingArguments(
    "distilbert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [121]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.016000,0.087596,0.919125,0.933985,0.926495,0.982266
2,0.022700,0.071123,0.921847,0.941563,0.931600,0.984856
3,0.012900,0.075370,0.930949,0.946783,0.938799,0.985651


TrainOutput(global_step=5268, training_loss=0.0166525886922328, metrics={'train_runtime': 344.355, 'train_samples_per_second': 122.324, 'train_steps_per_second': 15.298, 'total_flos': 460942247417400.0, 'train_loss': 0.0166525886922328, 'epoch': 3.0})

In [122]:
trainer.save_model("my_saved_model")

In [125]:
from transformers import pipeline

ner = pipeline(
    "token-classification",
    model = "my_saved_model",
    aggregation_strategy = "simple",
    device =0
)

In [126]:
s = "Bill Gates was the CEO of Microsofr in Seattle, Washington"

In [127]:
ner(s)

[{'entity_group': 'PER',
  'score': 0.9998944,
  'word': 'Bill Gates',
  'start': 0,
  'end': 10},
 {'entity_group': 'ORG',
  'score': 0.99988097,
  'word': 'Microsofr',
  'start': 26,
  'end': 35},
 {'entity_group': 'LOC',
  'score': 0.9996099,
  'word': 'Seattle',
  'start': 39,
  'end': 46},
 {'entity_group': 'LOC',
  'score': 0.9998123,
  'word': 'Washington',
  'start': 48,
  'end': 58}]